<center>
    <h1> Braitenberg Simulations </h1>
</center>

adopted from Vineel Nagisetty from his excellent code in https://github.com/vin-nag/braitenberg_vehicles/blob/master/Worksheet.ipynb

This tutorial is based on Doug Blank's "Jyro" module available for Jupyter for simulation on Braitenberg vehicles. Please note that the simulation as well as code were taken from [here](https://github.com/Calysto/jyro/blob/master/docs/source/RobotFindLight.ipynb/) although it was modified to suit this application and for improved readability. Please note that you need jyro module installed to run: `pip3 install --user jyro`

### Housekeeping

Basic imports and define constants

In [1]:
# Imports
from jyro.simulator import *

# Define constants
ONE = False
EXPLORER = False
INHIBITION_ONE = False
INHIBITION_TWO = False
CROSS_CONNECTED = False

### Define Environment and Robot

The environment is a simple square box with a light at the center. The robot is a 'Pioneer' jyro robot with light sensonrs as well as a camera which we use to view the simulation.

In [2]:
def make_environment(physics):
    """This function makes the background environment"""
    
    # create canvas object
    physics.addBox(0, 0, 10, 10, fill="backgroundgreen", wallcolor="black")
    
    # create light source
    physics.addLight(5, 5, 1.0) 

In [3]:
def make_robot(pose, max_range=5):
    """This function creates the robot"""
    
    # create a Pioneer robot
    robot = Pioneer("Pioneer", pose[0], pose[1], pose[2])
    
    # define and add light sensors
    light_sensors = PioneerFrontLightSensors(max_range) 
    robot.addDevice(light_sensors)
    robot.addDevice(Camera())
    return robot

### Define Controllers for Braitenberg Vehicles

There is one Braitenberg vehicle controller, with separate functions for one and two sensorimotors.

In [4]:
def braitenberg_controller(robot):
    """This function is the processor for Braitenberg vehicles"""
    
    # get light data
    light_data = robot["light"].getData()
    
    if ONE:
        # determine movement for Braitenberg one sensor vehicle
        translation, rotation = determine_braitenberg_one_move(light_data)
    else:
        # determine movement for Braitenberg one sensor vehicle
        translation, rotation = determine_braitenberg_two_move(light_data)
    
    robot.move(translation, rotation)   

In [5]:
def determine_braitenberg_one_move(light_data):
    """Returns tuple of (translation, rotation) movement for Braitenberg vehicle with one sensor and motor"""

    # determine speed
    max_light = max(light_data)
    
    # keep speed in range [0.1, 0.9]
    speed = 0.9 if max_light > 0.9 else max(0.1, max_light)
    
    if INHIBITION_ONE:
        speed = 1 - speed
    
    # determine angle
    left, right = light_data[0], light_data[1]
    difference = abs(left-right)
    rotation = 0

    # turn left or right depending on the direction of light source
    if difference > 0:
        rotation = 0.25 if left > right else -0.25
    
    return (speed, rotation)

In [6]:
def determine_braitenberg_two_move(light_data):
    """Returns tuple of (speed, rotation) movement for Braitenberg vehicle with two sensors and motors"""

    # determine speed
    max_light = min(light_data)
    
    # keep speed in range [0.1, 0.9]
    speed = 0.9 if max_light > 0.9 else max(0.1, max_light)
    
    # if sensors are inhibitory
    if INHIBITION_TWO:
        speed = 1 - speed

    # determine angle
    left, right = light_data[0], light_data[1]
    difference = abs(left-right)
    rotation = 0

    # turn left or right depending on the direction of light source
    if difference > 0:
        rotation = -15*difference if left > right else 15*difference

    # scale rotation speed down for explorer
    if EXPLORER:
        rotation /= 15
        
    # if sensors are connected to the opposite motors
    if CROSS_CONNECTED:
        rotation = -1 * rotation
        
    if INHIBITION_TWO:
        rotation = -1 * rotation
        
    return (speed, rotation)

### Simulate Braitenberg Vehicles

We now simulate six different Braitenberg vehicles, using jyro module. You can view the simulation on the left box and the robot's view on the right. Feel free to move the robot around using the verticle and horizontal sliders, updating the robot's heading using the slider under the box on the right. Click on play, pause and stop to run, pause and reset the simulation respectively. 

In [7]:
# select braitenberg one controller
ONE = True

# create the robot
robot = make_robot([3,2,0])

# connect the control
robot.brain = braitenberg_controller

# set connection to positive
INHIBITION_ONE = False

# create the simulation
simulator = VSimulator(robot, make_environment)

"""The first simulation is of a vehicle with one sensor and motor, where the motor goes faster the closer it is to the light source and goes towards it always."""

'The first simulation is of a vehicle with one sensor and motor, where the motor goes faster the closer it is to the light source and goes towards it always.'

In [8]:
# select braitenberg one controller
ONE = True

# create the robot
robot = make_robot([3,2,0])

# connect the control
robot.brain = braitenberg_controller

# set sensor to be negative
INHIBITION_ONE = True

# create the simulation
simulator = VSimulator(robot, make_environment)

"""The second simulation is of a vehicle with one sensor and motor, where the motor goes slower the closer it is to the light source and goes towards it always."""

'The second simulation is of a vehicle with one sensor and motor, where the motor goes slower the closer it is to the light source and goes towards it always.'

In [9]:
# select braitenberg two controller
ONE = False

# create the robot
robot = make_robot([1,1,0])

# connect the control
robot.brain = braitenberg_controller

# set sensors to negative and connections to the motors on the same side as the sensor
INHIBITION_TWO = False
CROSS_CONNECTED = False

# create the simulation
simulator = VSimulator(robot, make_environment)

"""The third simulation is of a vehicle with two sensors and motors, where the motor goes faster the closer it is to the light source and the sensors are connected to the motors on the same side. An observer might call this agent as having 'fear' towards the light source."""

"The third simulation is of a vehicle with two sensors and motors, where the motor goes faster the closer it is to the light source and the sensors are connected to the motors on the same side. An observer might call this agent as having 'fear' towards the light source."

In [10]:
# select braitenberg two controller
ONE = False

# create the robot
robot = make_robot([1,1,0])

# connect the control
robot.brain = braitenberg_controller

# set sensors to positive and connect to motors on the opposite side
INHIBITION_TWO = False
CROSS_CONNECTED = True

# create the simulation
simulator = VSimulator(robot, make_environment)

"""The fourth simulation is of a vehicle with two sensors and motors, where the motor goes faster the closer it is to the light source and the sensors are connected to the motors on the opposite side. An observer might call this agent as being 'aggressive' towards the light source."""

"The fourth simulation is of a vehicle with two sensors and motors, where the motor goes faster the closer it is to the light source and the sensors are connected to the motors on the opposite side. An observer might call this agent as being 'aggressive' towards the light source."

In [ ]:
# select braitenberg two controller
ONE = False

# create the robot
robot = make_robot([1,1,0])

# connect the control
robot.brain = braitenberg_controller

# set the appropriate controls
INHIBITION_TWO = True
CROSS_CONNECTED = False

# create the simulation
simulator = VSimulator(robot, make_environment)

"""The fifth simulation is of a vehicle with two sensors and motors, where the motor goes slower the closer it is to the light source and the sensors are connected to the motors on the same side. An observer might call this as agent as having 'love' towards the light source."""

In [ ]:
# select braitenberg two controller
ONE = False
EXPLORER = True

# create the robot
robot = make_robot([1,1,0])

# connect the control
robot.brain = braitenberg_controller

# set the appropriate controls
INHIBITION_TWO = True
CROSS_CONNECTED = True

# create the simulation
simulator = VSimulator(robot, make_environment)

"""The final simulation is of a vehicle with two sensors and motors, where the motor goes slower the closer it is to the light source and the sensors are connected to the motors on the opposite side. An observer might call this agent an 'Explorer'."""